In [9]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import glob
import json

from skimage import draw

import matplotlib.pyplot as plt

import os
from cv2 import cv2

from PIL import Image

import random

import tifffile as tiff

import pandas as pd

import cv2
import numpy as np
# import openslide
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from tifffile import memmap

import albumentations as A

from skimage import morphology as morph

import skimage
from skimage.color import rgb2gray, rgb2hed, hed2rgb, rgb2hsv, hsv2rgb
from skimage.exposure import rescale_intensity
from skimage.util import dtype

import torch
import torch.nn.functional as F

import torchvision
from torchvision import transforms

import logging

import warnings
warnings.filterwarnings("ignore")

from SwinUnet.networks.vision_transformer import SwinUnet as ViT_seg

from scipy.ndimage import measurements
from skimage.morphology import remove_small_objects

import segmentation_models_pytorch as smp

import numpy as np
import os
import pandas as pd
from tqdm.auto import tqdm

from metrics.stats_utils import get_pq, get_multi_pq_info, get_multi_r2

from scipy.ndimage import filters, measurements
from scipy.ndimage.morphology import (
    binary_dilation,
    binary_fill_holes,
    distance_transform_cdt,
    distance_transform_edt,
)

from skimage.segmentation import watershed

import sys
# sys.path.append('segmenter/')

# from segmenter.segm.model.factory import create_segmenter


from my_misc import *
from my_misc import __proc_np_hv


import pickle


def process_segmentation(output_binary, output_hv, output_classification):

    np_map = torch.softmax(output_binary, 1)[0].cpu().detach().permute(1, 2, 0)[..., 1].unsqueeze(2).numpy()
    np_map = cv2.resize(np_map.astype('float32'), (0, 0), fx=2.0, fy=2.0)

    hv_map = output_hv[0].cpu().detach().permute(1, 2, 0).numpy().astype('float32')
    hv_map = cv2.resize(hv_map, (0, 0), fx=2.0, fy=2.0) 

    tp_map = torch.argmax(torch.softmax(output_classification, 1), 1).cpu().numpy()[0]
    tp_map = cv2.resize(tp_map, (0, 0), fx=2.0, fy=2.0, interpolation=cv2.INTER_NEAREST).astype('float32')

    inst_map = __proc_np_hv(np_map[..., None], hv_map)
    inst_dict = get_instance_info(inst_map, tp_map)

    type_map = np.zeros_like(inst_map)
    inst_type_colours = np.array([
        [v['type']] * 3 for v in inst_dict.values()
    ])
    type_map = overlay_prediction_contours(
        type_map, inst_dict,
        line_thickness=-1,
        inst_colours=inst_type_colours)

    pred_map = np.dstack([inst_map, type_map])

    pred_map = cv2.resize(pred_map, (0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_NEAREST)
    
    return pred_map





def process_segmentation_2branch(output_binary, output_classification):

    tp_map = torch.argmax(torch.softmax(output_classification, 1), 1).cpu().numpy()[0]

    temp = torch.argmax(torch.softmax(output_binary, 1), 1).cpu()[0]
    inst_map = measurements.label(temp)[0]
    inst_map = remove_small_objects(inst_map, 10)
    
    inst_dict = get_instance_info(inst_map, tp_map)

    type_map = np.zeros_like(inst_map)
    inst_type_colours = np.array([
        [v['type']] * 3 for v in inst_dict.values()
    ])
    type_map = overlay_prediction_contours(
        type_map, inst_dict,
        line_thickness=-1,
        inst_colours=inst_type_colours)

    pred_map = np.dstack([inst_map, type_map])

    
    return pred_map



class ConicDataset(Dataset):

    def __init__(self, image_path, mask_path, counts, source, transform, copypaste, copy_level):

        self.image_path = image_path
        self.mask_path = mask_path
        self.counts = counts
        self.source = source

        self.transform = transform
        self.copypaste = copypaste
        self.copy_level = copy_level
        
        
#         with open('insufficient_class_alldict/insufficient_class_alldict.pickle','rb') as temp:
#             self.insufficient_class_alldict = pickle.load(temp)  
        
#         with open('insufficient_class_alldict/insufficient_class_alldict_consep.pickle','rb') as temp:
#             self.insufficient_class_alldict_consep = pickle.load(temp)      
            
#         with open('insufficient_class_alldict/insufficient_class_alldict_crag.pickle','rb') as temp:
#             self.insufficient_class_alldict_crag = pickle.load(temp) 
            
#         with open('insufficient_class_alldict/insufficient_class_alldict_dpath.pickle','rb') as temp:
#             self.insufficient_class_alldict_dpath = pickle.load(temp) 
            
#         with open('insufficient_class_alldict/insufficient_class_alldict_glas.pickle','rb') as temp:
#             self.insufficient_class_alldict_glas = pickle.load(temp)       
            
#         with open('insufficient_class_alldict/insufficient_class_alldict_pannuke.pickle','rb') as temp:
#             self.insufficient_class_alldict_pannuke = pickle.load(temp)             
            
   

    def __len__(self):
        return (len(self.image_path))

    def __getitem__(self, idx):

        image = self.image_path[idx].astype('uint8')
        original = image.copy()

        mask = self.mask_path[idx]
        
        mask_instance = mask[..., 0]
        mask_class = mask[..., 1]
        mask_class_original = mask_class.copy()
        
#         counts = self.counts[idx]
#         source = self.source[idx]
        
#         default   
        class1_rate = 0.3
        class2_rate = 0.1
        class3_rate = 0.05
        class4_rate = 0.15
        class5_rate = 0.3
        class6_rate = 0.1

#         class1_rate = 0.4
#         class2_rate = 0
#         class3_rate = 0
#         class4_rate = 0.2
#         class5_rate = 0.4
#         class6_rate = 0
        
        resize_rate = 0
        
        self.nooverlap = False
        
        
        if self.copypaste:
            
            if self.copy_level == 'random':
                dict_ = self.insufficient_class_alldict
                
                if random.random() > 0.5:
                    
                    print('Copy Paste !')
                    
                    copypaste_result = copypaste_candidate_instance(image, mask_instance, mask_class, 
                                                                 class1_rate, class2_rate, class3_rate, 
                                                                 class4_rate, class5_rate, class6_rate, dict_, 
                                                                 resize_rate, self.nooverlap)

                    image = copypaste_result['pasted_image']
                    mask_instance = copypaste_result['pasted_mask_instance']
                    mask_class = copypaste_result['pasted_mask_class']
              
            
            elif self.copy_level == 'source':
                
                if source == 'consep':
                    dict_ = self.insufficient_class_alldict_consep
                    
                if source == 'crag':
                    dict_ = self.insufficient_class_alldict_crag            
                    
                if source == 'dpath':
                    dict_ = self.insufficient_class_alldict_dpath          
                    
                if source == 'glas':
                    dict_ = self.insufficient_class_alldict_glas
                    
                if source == 'pannuke':
                    dict_ = self.insufficient_class_alldict_pannuke                    
                
                if random.random() > 0.5:
                    
                    print('Copy Paste !')
                    
                    copypaste_result = copypaste_candidate_instance(image, mask_instance, mask_class, 
                                                                 class1_rate, class2_rate, class3_rate, 
                                                                 class4_rate, class5_rate, class6_rate, dict_, 
                                                                 resize_rate, self.nooverlap)

                    image = copypaste_result['pasted_image']
                    mask_instance = copypaste_result['pasted_mask_instance']
                    mask_class = copypaste_result['pasted_mask_class']
                    
            else:
                pass
                    
                    
                    
                    
                                      
                        
        if self.transform:
            
            image = A.ColorJitter(brightness=0.3, contrast=0.4, saturation=0.2, hue=0.2, p=0.5)(image=image)['image']

            image = A.OneOf([
                             A.GaussianBlur(blur_limit=(1, 3), p=1),
                             A.MedianBlur(blur_limit=3, p=1),
                             A.GaussNoise (var_limit=(10.0, 50.0), p=1)
                             ], p=0.5)(image=image)['image']
            
            
            output = A.Compose([
                A.ShiftScaleRotate(shift_limit=0, scale_limit=0, rotate_limit=360, interpolation=0, border_mode=4, p=0.5),
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.CoarseDropout(max_holes=4, max_height=40, max_width=40, 
                                min_holes=2, min_height=20, min_width=20, p=0.5)],
                #A.RandomRotate90(90, p=0.5)],
                additional_targets={'mask1' : 'image',
                                    'mask2' : 'image'})(image=image.astype('uint8'), 
                                                        mask1=mask_class.astype('uint8'),
                                                        mask2=mask_instance.astype('uint8'))            
            
            
            
            image = output['image']
            mask = output['mask1']
            mask_instance = output['mask2']            
            
            mask_instance = remap_label(mask_instance)
            hv_map = gen_targets(mask_instance, (256, 256))['hv_map']


            image = transforms.ToTensor()(image)
            #image = cropping_center_torch(image, crop_shape=(224, 224), batch=False)
            
            mask = torch.from_numpy(mask).unsqueeze(0)
            mask_instance = torch.from_numpy(mask_instance).unsqueeze(0)
            #mask = cropping_center_torch(mask, crop_shape=(224, 224), batch=False)
            
            hv_map = transforms.ToTensor()(hv_map)
            #hv_map = cropping_center_torch(hv_map, crop_shape=(224, 224), batch=False)
        
        
    
            return {"original" : original, "image": image, "mask": mask, "mask_instance" : mask_instance, 'hv_map' : hv_map, "mask_class_original" : mask_class_original}

        

        else:

            image_hflip = A.HorizontalFlip(p=1)(image=image)['image']
            image_hflip = transforms.ToTensor()(image_hflip)
            #image_hflip = cropping_center_torch(image_hflip, crop_shape=(224, 224), batch=False)
            
            image_vflip = A.VerticalFlip(p=1)(image=image)['image']
            image_vflip = transforms.ToTensor()(image_vflip)
            #image_vflip = cropping_center_torch(image_vflip, crop_shape=(224, 224), batch=False)
            
            image = transforms.ToTensor()(image)
            #image = cropping_center_torch(image, crop_shape=(224, 224), batch=False)
            
            mask = torch.from_numpy(mask_class).unsqueeze(0)
            #mask = cropping_center_torch(mask, crop_shape=(224, 224), batch=False)
            
            hv_map = gen_targets(mask_instance, (256, 256))['hv_map']
            hv_map = transforms.ToTensor()(hv_map)
            #hv_map = cropping_center_torch(hv_map, crop_shape=(224, 224), batch=False)
            
            return {"original" : original, "image": image, "mask": mask, 'hv_map' : hv_map, "image_hflip" : image_hflip, "image_vflip" : image_vflip,  'mask_instance' : mask_instance,  "mask_class_original" : mask_class_original}  


        
        
        
        
        

from datetime import datetime

year = datetime.today().year
month = datetime.today().month
day = datetime.today().day  

today = str(year) + str(month) + str(day)        
        
        
        
        
        
gamma = 1

run_info = '-'.format(today, gamma)

if not os.path.exists('checkpoint/{}'.format(run_info)):
    os.mkdir('checkpoint/{}'.format(run_info))

log = logging.getLogger('staining_log')
log.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
fileHandler = logging.FileHandler('checkpoint/{}/log.txt'.format(run_info))
streamHandler = logging.StreamHandler()
fileHandler.setFormatter(formatter)
streamHandler.setFormatter(formatter)
#
log.addHandler(fileHandler)
log.addHandler(streamHandler)

In [6]:
train_images = np.load('final_data/train_imgs.npy')
train_masks = np.load('final_data/train_anns.npy')

test_images = np.load('final_data/valid_imgs.npy')
test_masks = np.load('final_data/valid_anns.npy')

In [10]:
train_dataset = ConicDataset(train_images, train_masks, None, None, transform=True, copypaste=False, copy_level = None)
train_dataloader = DataLoader(train_dataset, batch_size=8, num_workers=12, shuffle=True)

test_dataset = ConicDataset(test_images, test_masks, None, None, transform=False, copypaste=False, copy_level = None)
test_dataloader = DataLoader(test_dataset, batch_size=1, num_workers=12, shuffle=False)

In [16]:
temp = train_dataset[0]#['image'].permute(1, 2, 0)

In [17]:
image = temp['image'].permute(1, 2, 0)
mask = temp['mask']